<a href="https://colab.research.google.com/github/jrojasquiroz/hppAI/blob/main/evacal/carpetas_evacal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Librerías y versiones

In [1]:
import os
import pandas as pd
import shutil
from PIL import Image
import random
import tensorflow as tf
from keras import layers,models
import numpy as np
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Importo `keras_version` para luego ver qué versión en específico de esa librería se está usando, y `sys` para ver la versión de Python.

In [5]:
from keras import __version__ as keras_version
import sys

Y ahora sí veo todas las versiones. Las bibliotecas estándar de Python, como `os` y `shutil`, no tienen atributos de versión (`__version__`), debido a que son parte del núcleo de Python y su versión corresponde a la versión de Python que se está usando.

In [6]:
library_versions = {
    "python": sys.version,
    "os": os.__version__ if hasattr(os, '__version__') else 'N/A',
    "pandas": pd.__version__,
    "shutil": shutil.__version__ if hasattr(shutil, '__version__') else 'N/A',
    "PIL": Image.__version__,
    "random": random.__version__ if hasattr(random, '__version__') else 'N/A',
    "tensorflow": tf.__version__,
    "keras": keras_version,
    "numpy": np.__version__,
    "cv2": cv2.__version__,
    "ImageDataGenerator": ImageDataGenerator.__module__
}

library_versions

{'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]',
 'os': 'N/A',
 'pandas': '2.0.3',
 'shutil': 'N/A',
 'PIL': '9.4.0',
 'random': 'N/A',
 'tensorflow': '2.15.0',
 'keras': '2.15.0',
 'numpy': '1.25.2',
 'cv2': '4.8.0',
 'ImageDataGenerator': 'keras.src.preprocessing.image'}

# Revisión de datos

En un inicio, había corrido este cuaderno con un error en el código: el número de imágenes en `orden_cuarto_intento` era mucho menor que la cantidad original, en `cuarto_intento/imagenes_nuevo`. Por ello, antes de pasar a rehacer todo el código, en las siguientes dos celdas me aseguro de contar cuántas tenía por cada carpeta.

In [ ]:
# Ruta base para las imágenes
ruta_base = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento"

def contar_imagenes(ruta_base):
    total_imagenes = 0

    # Recorrer todas las carpetas y subcarpetas en ruta_base
    for root, dirs, files in os.walk(ruta_base):
        for file in files:
            # Verificar si el archivo es una imagen (jpg o png)
            if file.endswith('.jpg') or file.endswith('.png'):
                total_imagenes += 1

    return total_imagenes

# Llamar a la función y mostrar el resultado
numero_total_imagenes = contar_imagenes(ruta_base)
print(f"El número total de imágenes en '{ruta_base}' es: {numero_total_imagenes}")

El número total de imágenes en '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento' es: 1972


In [ ]:
# Ruta base para las imágenes
ruta_base = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/imagenes_nuevo"

def contar_imagenes(ruta_base):
    total_imagenes = 0

    # Recorrer todas las carpetas y subcarpetas en ruta_base
    for root, dirs, files in os.walk(ruta_base):
        for file in files:
            # Verificar si el archivo es una imagen (jpg o png)
            if file.endswith('.jpg') or file.endswith('.png'):
                total_imagenes += 1

    return total_imagenes

# Llamar a la función y mostrar el resultado
numero_total_imagenes = contar_imagenes(ruta_base)
print(f"El número total de imágenes en '{ruta_base}' es: {numero_total_imagenes}")

El número total de imágenes en '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/imagenes_nuevo' es: 14511


Una vez que ya estoy seguro de que había un error (1972<14511), procedo con el código, siendo consciente de que debo corregir eso más adelante.

# Inicio

Con el siguiente código creo carpetas tal y como lo he pensado: una por cada columna, y dentro de ellas `train` y `test`, similar [al ejemplo](https://www.youtube.com/watch?v=KZwuhl4hXok&t=83s) que he revisado.

En este caso, las creo dentro de `orden_cuarto_intento`.

In [ ]:
# Ruta base
ruta_base = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento"

# Lista de carpetas y subcarpetas
carpetas = {
    "C_contempo": ["train", "test"],
    "C_estado": ["train", "test"],
    "C_armarios": ["train", "test"],
    "B_contempo": ["train", "test"],
    "B_estado": ["train", "test"],
    "B_lavamano": ["train", "test"],
    "R_contempo": ["train", "test"],
    "R_estado": ["train", "test"],
    "R_carpinteria": ["train", "test"],
    "R_singular": ["train", "test"],
    "R_ventana": ["train", "test"]
}

# Crear las carpetas y subcarpetas
for carpeta, subcarpetas in carpetas.items():
    carpeta_path = os.path.join(ruta_base, carpeta)
    os.makedirs(carpeta_path, exist_ok=True)
    for subcarpeta in subcarpetas:
        subcarpeta_path = os.path.join(carpeta_path, subcarpeta)
        os.makedirs(subcarpeta_path, exist_ok=True)
        # Crear subcarpetas adicionales dentro de train y test
        if carpeta == "C_estado" or carpeta == "B_estado" or carpeta == "R_estado":
            for i in range(3):  # Crear subcarpetas del 0 al 2
                subcarpeta_adicional_path = os.path.join(subcarpeta_path, str(i))
                os.makedirs(subcarpeta_adicional_path, exist_ok=True)
        elif carpeta == "C_contempo" or carpeta == "B_contempo" or carpeta == "R_contempo":
            for i in range(4):  # Crear subcarpetas del 0 al 3
                subcarpeta_adicional_path = os.path.join(subcarpeta_path, str(i))
                os.makedirs(subcarpeta_adicional_path, exist_ok=True)
        else:
            subcarpeta_adicional_path = os.path.join(subcarpeta_path, "0")
            os.makedirs(subcarpeta_adicional_path, exist_ok=True)
            subcarpeta_adicional_path = os.path.join(subcarpeta_path, "1")
            os.makedirs(subcarpeta_adicional_path, exist_ok=True)

Ahora voy a limpiar el dataframe que tiene la evaluación de cada caso.

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/data/v-muestra_rmb-abril24-responsables.xlsx')
print(df.columns.tolist())

['ID', 'Title', 'Link', 'precio_euros', 'codigo_inmueble', 'Description', 'last_mod', 'dummy_indice', 'indice_alquiler', 'area', 'n_rooms', 'price_m2', 'n_baths', 'n_aseo', 'laundry', 'buhardilla', 'despacho', 'trastero', 'terrace', 'kwhm2_year', 'kgCO2m2_year', 'dummy_consum_EPC', 'dummy_emission_EPC', 'consum_A', 'consum_B', 'consum_C', 'consum_D', 'consum_E', 'consum_F', 'consum_G', 'emission_A', 'emission_B', 'emission_C', 'emission_D', 'emission_E', 'emission_F', 'emission_G', 'AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'barrio', 'codigo_inmueble_numero', 'Lat_Y', 'Lon_X', 'multifam', 'atico', 'duplex', 'estudio', 'loft', 'piso', 'triplex', 'invalido', 'prob_sin_foto', 'qprecio', 'MUN', 'qprecio_municipio', 'C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana', 'dumi_precio_reduccion', 'Precio_reduccion', 'Anuncio_invalido', 'Responsable']


In [ ]:
df = df[(df['Anuncio_invalido'] == 0) & (df['Responsable'] != "TS")]

In [ ]:
df=df[['ID','C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
       'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']]

In [ ]:
df = df.dropna()

In [ ]:
len(df)

1372

In [ ]:
df[['C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
       'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']]=df[['C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
       'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']].astype(int)

Hay que tener cuidado que `df` siempre tenga esta apariencia. Si, por ejemplo, en lugar de valores enteros hubieran decimales, la operación hacia adelante puede arrojar errores.

Otra de las situaciones que observé fue que habían algunas columnas habían sido dejadas en blanco por error durante el llenado de datos.

In [ ]:
df.head()

,ID,C_contempo,C_estado,C_armarios,B_contempo,B_estado,B_lavamano,R_contempo,R_estado,R_carpinteria,R_singular,R_ventana
7,8,1,2,1,1,2,1,1,1,1,0,0
8,9,1,2,1,1,2,1,3,2,1,1,0
11,12,2,2,0,1,2,1,2,2,1,0,0
16,17,1,2,0,1,2,1,1,2,1,0,0
18,19,1,2,1,1,2,1,1,2,1,0,1


A continuación, dado que en `cuarto_intento/imagenes_nuevo` es en donde están almacenadas las imágenes tal y como los chicos las descargaron, reviso esa carpeta y correspondientes sub-carpetas para trasladarlas hacia `orden_cuarto_intento`.

In [ ]:
# Ruta base para las imágenes
imagenes_base = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/imagenes_nuevo"

# Ruta base para las imágenes modificadas
imagenes_mod = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento"

# Contador de imágenes procesadas
imagenes_procesadas = 0

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    caso_id = str(row['ID'])  # Obtener el ID del caso

    # Iterar sobre las letras (B, C, R)
    for letra in ['B', 'C', 'R']:
        carpeta_caso = os.path.join(imagenes_base, caso_id, letra)

        # Verificar si la carpeta_caso existe
        if not os.path.exists(carpeta_caso):
            continue  # Si la carpeta no existe, pasar al siguiente ciclo

        # Iterar sobre las columnas relevantes
        for columna in ['C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
                        'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']:
            valor = row[columna]
            # Determinar la ruta de destino según la calificación
            ruta_destino = os.path.join(imagenes_mod, columna, 'train', str(valor))

            # Verificar si la ruta de destino existe, si no, crearla
            if not os.path.exists(ruta_destino):
                os.makedirs(ruta_destino)

            # Copiar las imágenes a la carpeta de destino
            for item in os.listdir(carpeta_caso):
                # Obtener la ruta completa del elemento
                item_path = os.path.join(carpeta_caso, item)
                # Verificar si el elemento es un archivo y si tiene la extensión .png o .jpg
                if os.path.isfile(item_path) and (item.endswith('.png') or item.endswith('.jpg')):
                    # Obtener la letra de la carpeta de origen
                    letra_origen = carpeta_caso.split(os.sep)[-1]
                    # Verificar si la letra de la carpeta de origen coincide con la letra de la columna
                    if letra_origen == columna.split('_')[0]:
                        try:
                            # Copiar el archivo a la carpeta de destino
                            shutil.copy(item_path, ruta_destino)
                            imagenes_procesadas += 1
                        except Exception as e:
                            print(f"Error al copiar {item_path} a {ruta_destino}: {e}")

print(f"Número total de imágenes procesadas: {imagenes_procesadas}")

Número total de imágenes procesadas: 58035


# 1. `B_contempo`

## 1.1. Muestras de testeo

Ahora corresponde mover imágenes desde `train` hacia `test`. Tomaré el 20% de cada categoría, `0`, `1`, `2` y `3` en este caso.


In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/test/2"

# Ruta de la carpeta de entrenamiento y prueba para la clase 3
train_folder_3 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/train/3"
test_folder_3 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/test/3"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

# Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 20% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

# Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 20% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 535
El 20% del número total de imágenes para la clase 0 es: 107.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 392
El 20% del número total de imágenes para la clase 1 es: 78.0


Me había olvidado de las clases 2 y 3, así que aquí van:

In [ ]:
# Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2, 0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, 20)

# Repetir el proceso para la clase 3
num_images_train_3 = len(os.listdir(train_folder_3))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 3:", num_images_train_3)

# Calcular el 20% de esa cantidad
percentage_20_train_3 = round(.2*num_images_train_3, 0)
print("El 20% del número total de imágenes para la clase 3 es:", percentage_20_train_3)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 3
move_random_images(train_folder_3, test_folder_3, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 2: 62
El 20% del número total de imágenes para la clase 2 es: 12.0
Número total de imágenes en la carpeta de entrenamiento para la clase 3: 47
El 20% del número total de imágenes para la clase 3 es: 9.0


## 1.2. Elaboración del modelo

Y ahora pruebo con una Red Neuronal Convulacional (CNN).

In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(4),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/mimodelo.keras')

Epoch 1/80
26/26 [==============================] - 7s 42ms/step - loss: 1.5574 - accuracy: 0.4277
Epoch 2/80
26/26 [==============================] - 1s 27ms/step - loss: 0.5279 - accuracy: 0.4494
Epoch 3/80
26/26 [==============================] - 1s 27ms/step - loss: 0.5019 - accuracy: 0.4783
Epoch 4/80
26/26 [==============================] - 1s 27ms/step - loss: 0.4947 - accuracy: 0.4855
Epoch 5/80
26/26 [==============================] - 1s 27ms/step - loss: 0.4783 - accuracy: 0.5048
Epoch 6/80
26/26 [==============================] - 1s 26ms/step - loss: 0.4745 - accuracy: 0.5193
Epoch 7/80
26/26 [==============================] - 1s 26ms/step - loss: 0.4811 - accuracy: 0.4602
Epoch 8/80
26/26 [==============================] - 1s 27ms/step - loss: 0.4668 - accuracy: 0.5000
Epoch 9/80
26/26 [==============================] - 1s 27ms/step - loss: 0.4582 - accuracy: 0.5253
Epoch 10/80
26/26 [==============================] - 1s 27ms/step - loss: 0.4588 - accuracy: 0.4964
Epoch 11/

## 1.3. Pruebas con los datos de testeo

Volveré a correr el modelo pero tomando los datos de la carpeta `test` para calcular el accuracy.

In [ ]:
# Definir el ancho y alto de las imágenes
width = 580
height = 580

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

7/7 [==============================] - 1s 48ms/step - loss: 2.0427 - accuracy: 0.5485
Accuracy en los datos de prueba: 0.5485436916351318


## 1.4. Modelo con image augmentation y normalización de colores

El código que viene a continuación (que se usa para todas las otras dimensiones) sí funciona, pero debido a que el tiempo de procesamiento era bastante alto y las diferencias en resultados con la red neuronal *local* no era mucha (lo hice con algunas dimensiones a manera de borrador), decidí no correrla más. Con más tiempo disponible se puede ejecutar.

Me queda <mark>pendiente</mark> ver si es correcto que se creen objetos x_train, y_train, x_test y y_test.

In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_contempo/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

# 2. `B_estado`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/test/2"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 20% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 20% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2,0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, 20)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 168
El 20% del número total de imágenes para la clase 0 es: 34.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 517
El 20% del número total de imágenes para la clase 1 es: 103.0
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 345
El 20% del número total de imágenes para la clase 2 es: 69.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(3),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

7/7 [==============================] - 0s 43ms/step - loss: 3.0393 - accuracy: 0.4585
Accuracy en los datos de prueba: 0.4585365951061249


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 100
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_estado/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

# 3. `B_lavamano`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 464
El 20% del número total de imágenes para la clase 0 es: 93.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 479
El 20% del número total de imágenes para la clase 1 es: 96.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 100

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

6/6 [==============================] - 0s 42ms/step - loss: 3.6348 - accuracy: 0.5843
Accuracy en los datos de prueba: 0.5843373537063599


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/B_lavamano/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

GPUs disponibles: 1
28/28 [==============================] - 5s 152ms/step - loss: 1.5394 - accuracy: 0.5067
Accuracy en los datos de prueba: 0.5067415833473206


# 4. `C_contempo`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/test/2"

# Ruta de la carpeta de entrenamiento y prueba para la clase 3
train_folder_3 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/train/3"
test_folder_3 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/test/3"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 20% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 20% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2,0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, percentage_20_train_2)

### Repetir el proceso para la clase 3
num_images_train_3 = len(os.listdir(train_folder_3))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 3:", num_images_train_3)

# Calcular el 20% de esa cantidad
percentage_20_train_3 = round(.2*num_images_train_3,0)
print("El 20% del número total de imágenes para la clase 3 es:", percentage_20_train_3)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 3
move_random_images(train_folder_3, test_folder_3, percentage_20_train_3)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 493
El 20% del número total de imágenes para la clase 0 es: 99.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 401
El 20% del número total de imágenes para la clase 1 es: 80.0
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 96
El 20% del número total de imágenes para la clase 2 es: 19.0
Número total de imágenes en la carpeta de entrenamiento para la clase 3: 54
El 20% del número total de imágenes para la clase 3 es: 11.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(4),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 100

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

7/7 [==============================] - 0s 45ms/step - loss: 2.8807 - accuracy: 0.4493
Accuracy en los datos de prueba: 0.4492753744125366


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 100
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_contempo/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

GPUs disponibles: 1
26/26 [==============================] - 4s 163ms/step - loss: 1.4684 - accuracy: 0.1986
Accuracy en los datos de prueba: 0.19855596125125885


# 5. `C_estado`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/test/2"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 20% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 20% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2,0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, percentage_20_train_2)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 114
El 20% del número total de imágenes para la clase 0 es: 23.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 511
El 20% del número total de imágenes para la clase 1 es: 102.0


ValueError: Sample larger than population or is negative

Como no entendía por qué sucedía el error, simplemente moví 102 archivos de train a test para la clasificación 1.

In [ ]:
### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2,0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, percentage_20_train_2)

Número total de imágenes en la carpeta de entrenamiento para la clase 2: 368
El 20% del número total de imágenes para la clase 2 es: 74.0


Y ahora ya paso a correr el modelo.

In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(3),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 100

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

7/7 [==============================] - 0s 43ms/step - loss: 2.9812 - accuracy: 0.4360
Accuracy en los datos de prueba: 0.4360189437866211


Prueba con <mark>estandarización de color</mark> e <mark>image augmentation</mark>.

In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 100
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_estado/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

GPUs disponibles: 1
15/15 [==============================] - 2s 143ms/step - loss: 1.1415 - accuracy: 0.3022
Accuracy en los datos de prueba: 0.302173912525177


# 6. `C_armarios`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 227
El 20% del número total de imágenes para la clase 0 es: 45.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 684
El 20% del número total de imágenes para la clase 1 es: 137.0


ValueError: Sample larger than population or is negative

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
#percentage_20_train_1 = round(num_images_train_1*.2,0)
#print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 1: 684


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

6/6 [==============================] - 0s 54ms/step - loss: 3.7444 - accuracy: 0.7198
Accuracy en los datos de prueba: 0.7197802066802979


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/C_armarios/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

GPUs disponibles: 1
8/8 [==============================] - 2s 229ms/step - loss: 0.8137 - accuracy: 0.5714
Accuracy en los datos de prueba: 0.5714285969734192


# 7. `R_contemporaneo`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/test/2"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_3 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/train/3"
test_folder_3 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/test/3"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_1)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2,0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, 20)

### Repetir el proceso para la clase 3
num_images_train_3 = len(os.listdir(train_folder_3))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 3:", num_images_train_1)

# Calcular el 20% de esa cantidad
percentage_20_train_3 = round(.2*num_images_train_3,0)
print("El 20% del número total de imágenes para la clase 3 es:", percentage_20_train_3)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 3
move_random_images(train_folder_3, test_folder_3, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 1234
El 20% del número total de imágenes para la clase 0 es: 247.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 831
El 20% del número total de imágenes para la clase 1 es: 166.0
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 831
El 20% del número total de imágenes para la clase 2 es: 59.0
Número total de imágenes en la carpeta de entrenamiento para la clase 3: 831
El 20% del número total de imágenes para la clase 3 es: 28.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(4),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

16/16 [==============================] - 1s 28ms/step - loss: 2.0703 - accuracy: 0.4337
Accuracy en los datos de prueba: 0.4337349534034729


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_contempo/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

GPUs disponibles: 1


KeyboardInterrupt: 

# 8. `R_estado`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/test/2"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = round(.2*num_images_train_2,0)
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 470
El 20% del número total de imágenes para la clase 0 es: 94.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 1263
El 20% del número total de imágenes para la clase 1 es: 253.0
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 685
El 20% del número total de imágenes para la clase 2 es: 137.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(3),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

16/16 [==============================] - 1s 24ms/step - loss: 2.3632 - accuracy: 0.4555
Accuracy en los datos de prueba: 0.455486536026001


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 70
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_estado/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

# 9. `R_singular`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 1720
El 20% del número total de imágenes para la clase 0 es: 344.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 388
El 20% del número total de imágenes para la clase 1 es: 78.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

14/14 [==============================] - 1s 27ms/step - loss: 2.1040 - accuracy: 0.8147
Accuracy en los datos de prueba: 0.8147268295288086


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_singular/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

# 10. `R_ventana`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 1170
El 20% del número total de imágenes para la clase 0 es: 234.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 1028
El 20% del número total de imágenes para la clase 1 es: 206.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

14/14 [==============================] - 1s 30ms/step - loss: 3.4512 - accuracy: 0.5239
Accuracy en los datos de prueba: 0.5239179730415344


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_ventana/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

# 11. `R_carpinteria`

In [ ]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = round(.2*num_images_train_0,0)
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, 20)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = round(.2*num_images_train_1,0)
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, 20)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 833
El 20% del número total de imágenes para la clase 0 es: 167.0
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 1398
El 20% del número total de imágenes para la clase 1 es: 280.0


In [ ]:
width = 580
height = 580

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

14/14 [==============================] - 1s 31ms/step - loss: 3.7866 - accuracy: 0.5708
Accuracy en los datos de prueba: 0.5707865357398987


In [ ]:
# Verificar si GPU está disponible
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

# Parámetros
width, height = 580, 580

# Rutas
ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/train/'
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/test/'

# Función para cargar y procesar imágenes
def cargar_imagenes(ruta):
    imagenes = []
    etiquetas = []
    labels = os.listdir(ruta)
    for i in os.listdir(ruta):
        for j in os.listdir(os.path.join(ruta, i)):
            img_path = os.path.join(ruta, i, j)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
            img = cv2.resize(img, (width, height))
            img = img / 255.0  # Normalización
            imagenes.append(img)
            array = np.zeros(len(labels))
            array[labels.index(i)] = 1
            etiquetas.append(array)
    return np.array(imagenes), np.array(etiquetas)

# Cargar datos
x_train, y_train = cargar_imagenes(ruta_train)
x_test, y_test = cargar_imagenes(ruta_test)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False
)
datagen.fit(x_train)

# Definir el modelo
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(len(os.listdir(ruta_train))),  # Número de clases
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo con Data Augmentation
epochs = 80
batch_size = 32

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, verbose=0)

# Guardar el modelo
model.save('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_cuarto_intento/R_carpinteria/mimodelo_version2.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)